# 用 PyTorch 自动求导

## 正向传播和反向传播

自动求导计算一个函数在指定值上时的导数。自动求导先会生成一个无环计算图，可以是现实构造（Tensorflow/Theano/MXNet）也可以是隐式构造（PyTorch/MXNet）。

根据链式法则，求导可以看成是中间部分各自求导再相乘。

自动求导有两种模式：正向传播，反向传播(back propogation)

正向传播从最内部包含 x 作为 input 的部分开始向外计算，反向传播则从最外层向内计算，使用正向计算函数时已经算好的中间值。机器学习中一般用反向传播。

### 反向传播
跟踪整改计算图，填充关于每个参数的偏导数。

#### 流程
1. 构造计算图
2. 前向执行计算图，存储中间结果
3. 反向执行图，计算导数
   1. 去除不需要的枝
   
#### 反向传播总结
计算图的操作子个数为 n。
- 计算复杂度：O(n)，与正向传播一样
- 内存复杂度：O(n)，因为需要存储
  - 正向传播为 O(1)，但每步都需要扫一遍全部计算图


## 自动求导的 PyTorch 实现



### 例：

对函数 y = 2x<sup>T</sup>x 关于列向量 x 求导。一个标量函数对向量求梯度，得到的是一个与之形状相同的向量。

根据点积求导公式

y = <u, v>, dy/d<b>x</b> = u<sup>T</sup> * (dv/dx) + v<sup>T</sup> * (du/dx)

梯度应为 4x

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

计算 y 关于 x 梯度前，需要有个地方存储梯度。可以用 `requires_grad` 参数在创建向量时准备（ `torch.arange(4.0, requires_grad=True)`），也可以在之后用 `x.requires_grad_(True)` 。

In [3]:
x.requires_grad_(True)
x.grad # 默认是 None

计算 y，是个标量。

In [5]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

反向传播，这步会自动计算 y 关于 x 向量的每个分量的梯度。于是我们就可以打印梯度 `x.grad`

In [6]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

PyTorch 默认会累积梯度。当要计算 x 的另一个函数的梯度，先要清除 x 现在的梯度 `x.grad.zero_()`。

In [8]:
x.grad.zero_()
x.grad

tensor([0., 0., 0., 0.])

In [9]:
# 计算另一个关于 x 的函数的梯度
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [10]:
# 再运行一次反向传播，可以看到梯度被累积了
y.backward()
x.grad

tensor([2., 2., 2., 2.])

#### 非标量 y

当 y 不是标量时，向量 y 关于向量 x 的导数会是一个矩阵。更高阶时则会是一个高阶张量。
此时计算 `backward()` 时要传入一个 `gradient` 参数。

机器学习中我们通常不是想计算高阶微分张量，而是想要求一个 batch 的训练样本中各个部分的损失函数的偏导数的和，所以可以传入一个全是 1 的向量作为 gradient 参数。

In [11]:
x.grad.zero_()
y = x * x
y.backward(torch.ones(len(x)))
# same as y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

#### 分离计算

用 `detach()` 将一部分计算分离作为常量看待，这样反向传播计算梯度时就不会考虑计算图的这个计算。

In [16]:
x.grad.zero_()
y = x * x

# u 成为一个储存 y 计算结果的常量向量
u = y.detach()
print('x:', x)
print('u:', u)

# z 是用一个常量向量 [0,1,4,9] 乘以 x
z = u * x
z.sum().backward()
x.grad
# x 的梯度与 u 相同

x: tensor([0., 1., 2., 3.], requires_grad=True)
u: tensor([0., 1., 4., 9.])


tensor([0., 1., 4., 9.])

### 用了Python控制流的梯度

In [23]:
# This eseentially returns c * x, 
# c is determined with complex while, if else flow
def f(x):
    b = torch.randn(size=())
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c * x

When size is set to an empty tuple (), as in torch.randn(size=()), it indicates that a tensor with a single element should be created. In other words, this creates a scalar tensor with a random value sampled from a standard normal distribution (i.e., mean 0 and standard deviation 1).

In [24]:
a = torch.randn(size=(), \
    requires_grad=True)
print('a:', a)

# 用了 Python 控制流的复杂的计算图
d = f(a)
print('d:', d)

d.backward()
a.grad, d/a


a: tensor(-0.0484, requires_grad=True)
d: tensor(-64.9551, grad_fn=<MulBackward0>)


(tensor(1341.9515), tensor(1341.9515, grad_fn=<DivBackward0>))